# Final Project Code

In [5]:
from urllib.request import urlopen
import json
import pandas as pd

import psycopg2
import psycopg2.extras

# Data Preprocessing

## Import Station Status Data

In [15]:
URL_STATION_INFO = "https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json"
with urlopen(URL_STATION_INFO) as url:
    data_station_status = json.load(url)

columns = ["station_id", "short_name", "name", "lat", "lon", "region_id", "capacity", "rental_uris"]
station_data = {column:[] for column in columns}
for station in data_station_status["data"]["stations"]:
    for column in columns:
        if column in station.keys():
            station_data[column].append(station[column])
        else:
            station_data[column].append(None)

df_station_status = pd.DataFrame(station_data)
df_station_status["rental_uris"] = df_station_status["rental_uris"].astype(str)
df_station_status.head()

,station_id,short_name,name,lat,lon,region_id,capacity,rental_uris
0,7d5fa1f0-4069-4d5d-b735-5d500597394a,5569.07,W Broadway & Watts St,40.723230,-74.003140,71,51,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
1,40043e8a-6ef6-428d-b585-a7c23cdb06d4,6098.12,Broadway & E 19 St,40.738661,-73.989873,71,65,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
2,19b61564-7629-41b4-80c8-1756135a5442,7913.15,Cauldwell Ave & E 158 St,40.819890,-73.908351,71,19,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
3,c1a4d909-0a00-475a-8e82-18ed13a4eb01,4962.02,Whitehall St & Bridge St,40.703662,-74.013181,71,44,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
4,7779e057-b33e-46a7-8a1e-8c28bcd9a558,4325.03,Thomas S. Boyland St & Macon St,40.684910,-73.914930,71,21,{'android': 'https://bkn.lft.to/lastmile_qr_sc...


# Database Upload

### Connect, Set up schema

In [16]:
# PSQL db connection using psycopg2
conn = psycopg2.connect(
    dbname='new_db', 
    user='awesome_user', 
    password='awesome_password', 
    host='postgres', 
    port='5432'
)
c = conn.cursor()

# Create new schema in new_db database
c.execute('CREATE SCHEMA IF NOT EXISTS final_project')
conn.commit()

### Clean all tables

In [20]:
# create table with appropriate columns
c.execute('DROP TABLE IF EXISTS final_project.station_info')
c.execute('''
    CREATE TABLE IF NOT EXISTS final_project.station_info (
        short_name VARCHAR PRIMARY KEY,
        station_id VARCHAR,
        name VARCHAR,
        lat FLOAT,
        lon FLOAT,
        region_id VARCHAR,
        capacity INT,
        rental_uris VARCHAR
    )'''
)


### Upload Station Info

In [21]:
# convert df to dict for easier insertion into PSQL
data_dict = df_station_status.to_dict(orient='records')

# insert data into ny_irs_data table in irs_data schema
columns = data_dict[0].keys()

query = 'INSERT INTO final_project.station_info ({}) VALUES %s'.format(','.join(columns))

values = [[value for value in data.values()] for data in data_dict]

psycopg2.extras.execute_values(c, query, values)
conn.commit()

### Confirm upload

In [22]:
pd.read_sql_query("SELECT * FROM final_project.station_info LIMIT 100", conn)

/tmp/ipykernel_179/1361281294.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("SELECT * FROM final_project.station_info LIMIT 100", conn)


,short_name,station_id,name,lat,lon,region_id,capacity,rental_uris
0,5569.07,7d5fa1f0-4069-4d5d-b735-5d500597394a,W Broadway & Watts St,40.723230,-74.003140,71,51,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
1,6098.12,40043e8a-6ef6-428d-b585-a7c23cdb06d4,Broadway & E 19 St,40.738661,-73.989873,71,65,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
2,7913.15,19b61564-7629-41b4-80c8-1756135a5442,Cauldwell Ave & E 158 St,40.819890,-73.908351,71,19,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
3,4962.02,c1a4d909-0a00-475a-8e82-18ed13a4eb01,Whitehall St & Bridge St,40.703662,-74.013181,71,44,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
4,4325.03,7779e057-b33e-46a7-8a1e-8c28bcd9a558,Thomas S. Boyland St & Macon St,40.684910,-73.914930,71,21,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
...,...,...,...,...,...,...,...,...
95,7662.13,5f63411c-91ce-4788-a567-d388cc812a15,Lexington Ave & E 127 St,40.805726,-73.936322,71,33,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
96,6001.01,6d766daf-9298-4059-a329-1818582d495f,Borden Ave & Review Ave,40.738663,-73.940938,71,19,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
97,5540.06,69717638-5c4a-47a7-bccb-3b42c81eb09f,56 Dr & 61 St,40.723680,-73.904580,71,19,{'android': 'https://bkn.lft.to/lastmile_qr_sc...
98,4190.06,1d32e684-602b-46b6-8c96-ab47b57a03f3,Grand Ave & Bergen St,40.678045,-73.962408,71,25,{'android': 'https://bkn.lft.to/lastmile_qr_sc...


### Close the connection

In [14]:
conn.close()

# Report Generation